# You are what you eat - Relating Demographic Data to Food Consumption Habits

In [28]:
#%matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns  # needed for heatmap
from os.path import join
plt.rcParams['figure.figsize'] = [7, 5]

In [29]:
# define paths for data
FOLDER_PATH = 'data'
PURCHASES_PATH = 'area_level_purchases'
OSWARD_GROCERY = 'year_osward_grocery.csv'
WARD_ATLAS = 'ward-atlas-data.csv'

ward_data_path = join(FOLDER_PATH, PURCHASES_PATH, OSWARD_GROCERY)
atlas_data_path = join(FOLDER_PATH, WARD_ATLAS)

In [30]:
# load the data sets
df_ward = pd.read_csv(ward_data_path, sep=',', index_col=0)
df_atlas = pd.read_csv(atlas_data_path, sep=',', encoding='iso-8859-1')

## Atlas dataset preprocessing

In [31]:
df_atlas

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Population and Age; Population Estimates and Projections; 2001,Population and Age; Population Estimates and Projections; 2006,Population and Age; Population Estimates and Projections; 2011,Population and Age; Population Estimates and Projections; 2013,Population and Age; Population Estimates and Projections; 2014,Population and Age; Population Estimates and Projections; 2015,...,Nat Insurance No. registrations to adult overseas nationals; NINo Registrations; 2007/2008,Nat Insurance No. registrations to adult overseas nationals; NINo Registrations; 2008/2009,Nat Insurance No. registrations to adult overseas nationals; NINo Registrations; 2009/2010,Nat Insurance No. registrations to adult overseas nationals; NINo Registrations; 2010/2011,Nat Insurance No. registrations to adult overseas nationals; NINo Registrations; 2011/2012,Election turnout; Turnout Borough election; 2006,Election turnout; Turnout Borough election; 2010,Election turnout; Turnout Borough election; 2014,Election turnout; Turnout Mayoral election; 2008,Election turnout; Turnout Mayoral election; 2012
0,Codes,New Code,Borough,Names,2001,2006,2011,2013,2014,2015,...,2007/2008,2008/2009,2009/2010,2010/2011,2011/2012,2006.0,2010.0,2014.000000,2008.0,2012.000000
1,00AA,E09000001,City of London,City of London,7350,7254,7400,7993,8043,8102,...,800,730,580,780,805,NaN,NaN,NaN,47.6,48.500000
2,00ABFX,E05000026,Barking and Dagenham,Abbey,10350,10239,12900,13806,14283,14739,...,845,855,1050,1370,1000,29.9,58.1,36.820000,35.6,25.688941
3,00ABFY,E05000027,Barking and Dagenham,Alibon,9350,9236,10450,10527,10563,10587,...,160,215,190,260,190,41.1,59.2,34.270000,34.4,20.347925
4,00ABFZ,E05000028,Barking and Dagenham,Becontree,11550,11238,11650,12155,12428,12688,...,265,295,350,400,320,32.4,60.2,36.840000,33.4,22.538211
5,00ABGA,E05000029,Barking and Dagenham,Chadwell Heath,9400,9387,10100,10261,10337,10400,...,130,145,135,160,155,36.2,61.4,36.500000,36.6,25.318814
6,00ABGB,E05000030,Barking and Dagenham,Eastbrook,10200,10082,10600,10701,10741,10768,...,100,140,135,120,130,54.1,63.0,36.520000,38.8,24.121469
7,00ABGC,E05000031,Barking and Dagenham,Eastbury,10200,10452,11700,11816,11861,11901,...,355,370,370,440,340,39.4,59.4,34.630000,33.0,21.514876
8,00ABGD,E05000032,Barking and Dagenham,Gascoigne,9500,10745,12550,14174,15018,15845,...,525,625,650,645,480,33.0,58.4,37.570000,34.5,25.592096
9,00ABGE,E05000033,Barking and Dagenham,Goresbrook,10400,10087,11350,11408,11437,11472,...,235,235,200,265,200,41.4,57.8,34.730000,33.8,20.506730


In [32]:
df_atlas.columns.values

array(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Population and Age; Population Estimates and Projections; 2001',
       'Population and Age; Population Estimates and Projections; 2006',
       'Population and Age; Population Estimates and Projections; 2011',
       'Population and Age; Population Estimates and Projections; 2013',
       'Population and Age; Population Estimates and Projections; 2014',
       'Population and Age; Population Estimates and Projections; 2015',
       'Population and Age; Population Estimates and Projections; 2016',
       'Population and Age; Population Estimates and Projections; 2018',
       'Population and Age; Population Estimates and Projections; 2023',
       'Population and Age; Population Estimates and Projections; 2028',
       'Population and Age; Age structure (percentage) - 2013; Aged 0-15',
       'Population and Age; Age structure (percentage) - 2013; Aged 16-64',
       'Population and Age; Age structure (percentage) -

In [33]:
print(f"Rows: {df_atlas.shape[0]}, Columns: {df_atlas.shape[1]} of df_atlas")

Rows: 629, Columns: 946 of df_atlas


The `df_atlas` data set looks very messy. Row 0 contains info about the data. For every ward and demographic marker yearly (from 2001 to 2006) values exist in columns. According to the [source](https://data.london.gov.uk/dataset/ward-profiles-and-atlas) this data set was collected in September 2015 with the most up-to-date information available. Thus, for many markers, the last available information dates to 2015 or even earlier (e.g. 2013). Our first step is to clean the data set from outdated columns and only keep the most up-to-date demographic markers.

As indicated by the title, we are interested in demographic markers which help classifying different classes of people. These are for example gender, religion, ethnicity, wealth, age. Thus, in the following, we drop columns which we deem not helpful in classifying people into groups. We acknowledge that this process can be rather biased, however, as the data set has 946 columns, we see the need to reduce the data set's complexity.

In [34]:
# clean age (columns after 2013 are projections, therefore keep 2013 columns)
cond = df_atlas.columns.str.startswith('Population and Age') & ~df_atlas.columns.str.contains('2013')
df_atlas = df_atlas.loc[:, ~cond]
# drop absolute population values
df_atlas = df_atlas.drop(['Population and Age; Population Estimates and Projections; 2013',
               'Population and Age; Aged 0-15; 2013',
               'Population and Age; Aged 16-64; 2013',
               'Population and Age; Aged 65+; 2013',
               'Population and Age; All ages; 2013',
               'Population and Age; Mean age; 2013',
               'Population and Age; Median age; 2013'], axis=1)
# delete population density
cond = df_atlas.columns.str.startswith('Area and Density')
df_atlas = df_atlas.loc[:, ~cond]
# delete the 18 ethnic groups for now, as they add complexity
cond = df_atlas.columns.str.startswith('Diversity; Ethnic Group 18 groups')
df_atlas = df_atlas.loc[:, ~cond]
# delete household language (of no interest) 
cond = df_atlas.columns.str.startswith('Diversity; Household Language - 2011 Census')
df_atlas = df_atlas.loc[:, ~cond]
# delete household composition data
cond = df_atlas.columns.str.startswith('Household composition')
df_atlas = df_atlas.loc[:, ~cond]
# delete household repossesion
cond = df_atlas.columns.str.startswith('Home repossessions')
df_atlas = df_atlas.loc[:, ~cond]
# delete outdated birth and death markers
cond = df_atlas.columns.str.startswith('Births and deaths')
df_atlas = df_atlas.loc[:, ~cond]
# delete outdated life expectancy
cond = df_atlas.columns.str.startswith('Life Expectancy')
df_atlas = df_atlas.loc[:, ~cond]
# delete outdated housing markers
cond = df_atlas.columns.str.startswith('Housing type and Tenure')
df_atlas = df_atlas.loc[:, ~cond]
cond = df_atlas.columns.str.startswith('House Prices')
df_atlas = df_atlas.loc[:, ~cond]
# delete tax related data (not of interest)
cond = df_atlas.columns.str.startswith('Dwellings and Council tax')
df_atlas = df_atlas.loc[:, ~cond]
# delete dwelling data which is not of interest
cond = df_atlas.columns.str.startswith('Property Type') | df_atlas.columns.str.startswith('Size of dwellings') | df_atlas.columns.str.startswith('Property build period')
df_atlas = df_atlas.loc[:, ~cond]
# delete outdated income data
cond = df_atlas.columns.str.startswith('Household Income') & ~df_atlas.columns.str.contains('2012/13')
df_atlas = df_atlas.loc[:, ~cond]
# outdated employment data
cond = df_atlas.columns.str.startswith('Employment')
df_atlas = df_atlas.loc[:, ~cond]
# delete benefits claimants and jobseekers allowance
cond = df_atlas.columns.str.startswith('Benefits claimants') | df_atlas.columns.str.startswith('Jobseekers Allowance')
df_atlas = df_atlas.loc[:, ~cond]
# delete out of work families
cond = df_atlas.columns.str.startswith('Out-of-Work Families')
df_atlas = df_atlas.loc[:, ~cond]
# outdated poverty
cond = df_atlas.columns.str.startswith('Children in Poverty')
df_atlas = df_atlas.loc[:, ~cond]
# outdated "intelligence" scores
cond = df_atlas.columns.str.startswith('GCSE')
df_atlas = df_atlas.loc[:, ~cond]
cond = df_atlas.columns.str.startswith('A-Level Point Scores')
df_atlas = df_atlas.loc[:, ~cond]
# pupil abscence not if interest
cond = df_atlas.columns.str.startswith('Pupil Absence')
df_atlas = df_atlas.loc[:, ~cond]
# individual crimes not of interest, only crime rate
cond = df_atlas.columns.str.startswith('Crime') # & ~df_atlas.columns.str.contains('Total crime rate; 2014/15')
df_atlas = df_atlas.loc[:, ~cond]
# further columns not of interest
cond = df_atlas.columns.str.startswith('Fires') | df_atlas.columns.str.startswith('Ambulance') \
| df_atlas.columns.str.startswith('Binge Drinking') | df_atlas.columns.str.startswith('Road Casualties') \
| df_atlas.columns.str.startswith('Air Emissions') | df_atlas.columns.str.startswith('Land Use') \
| df_atlas.columns.str.startswith('Access to green space and nature') | df_atlas.columns.str.startswith('Public Transport Accessibility')\
| df_atlas.columns.str.startswith('Car access') | df_atlas.columns.str.startswith('Travel to work by bicycle') \
| df_atlas.columns.str.startswith('Workplace employment') | df_atlas.columns.str.startswith('Nat Insurance No. registrations') \
| df_atlas.columns.str.startswith('Election turnout')
df_atlas = df_atlas.loc[:, ~cond]
# obesity already available in cleaned format
cond = df_atlas.columns.str.startswith('Childhood Obesity Prevalence')
df_atlas = df_atlas.loc[:, ~cond]
# delete old indices of deprivation
cond = df_atlas.columns.str.startswith('Indices of Deprivation') & ~df_atlas.columns.str.contains('2010')
df_atlas = df_atlas.loc[:, ~cond]

The indices of deprivation measure how deprived an area is [source](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/464597/English_Indices_of_Deprivation_2015_-_Research_Report.pdf). In essence, these indicators indicate how impoverished an area is. The linked documents lists the meaning of the different indices. *The average score measure summarises the average level of deprivation across the higher-level area,based on the scores of the Lower-layer Super Output Areas in the area.* Therefore, we decided to keep only this measure, as we are interested in an indicator of "average impoverishment".

In [35]:
cond = df_atlas.columns.str.startswith('Indices of Deprivation') & ~df_atlas.columns.str.endswith('Average Score; 2010')
df_atlas = df_atlas.loc[:, ~cond]

In [36]:
df_atlas.columns.values

array(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Population and Age; Age structure (percentage) - 2013; Aged 0-15',
       'Population and Age; Age structure (percentage) - 2013; Aged 16-64',
       'Population and Age; Age structure (percentage) - 2013; Aged 65+',
       'Diversity; Ethnic Group 5 groups - 2011 Census; White',
       'Diversity; Ethnic Group 5 groups - 2011 Census; Mixed',
       'Diversity; Ethnic Group 5 groups - 2011 Census; Asian or Asian British',
       'Diversity; Ethnic Group 5 groups - 2011 Census; Black or Black British',
       'Diversity; Ethnic Group 5 groups - 2011 Census; Other',
       'Diversity; Religion - 2011 Census; Christian',
       'Diversity; Religion - 2011 Census; Buddhist',
       'Diversity; Religion - 2011 Census; Hindu',
       'Diversity; Religion - 2011 Census; Jewish',
       'Diversity; Religion - 2011 Census; Muslim',
       'Diversity; Religion - 2011 Census; Sikh',
       'Diversity; Religion - 2011 Census; O

In [37]:
print(f"Rows: {df_atlas.shape[0]}, Columns: {df_atlas.shape[1]} of df_atlas")

Rows: 629, Columns: 37 of df_atlas


In [38]:
df_atlas.head(3)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Population and Age; Age structure (percentage) - 2013; Aged 0-15,Population and Age; Age structure (percentage) - 2013; Aged 16-64,Population and Age; Age structure (percentage) - 2013; Aged 65+,Diversity; Ethnic Group 5 groups - 2011 Census; White,Diversity; Ethnic Group 5 groups - 2011 Census; Mixed,Diversity; Ethnic Group 5 groups - 2011 Census; Asian or Asian British,...,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 2 qualifications,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Apprenticeship,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 3 qualifications,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 4 qualifications and above,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Other qualifications,Qualifications; Qualifications and Students - 2011 Census; % of 16+ who are schoolchildren and full-time students: Age 18 and over,Health and Disability; General Health - 2011 Census; % People with Bad or Very Bad Health,Health and Disability; General Health - 2011 Census; % People whose Day-to-day activities are limited a lot,"Happiness and Well-being; Subjective well-being average score, 2011/12 and 2012/13;",Indices of Deprivation; Average Score; 2010
0,Codes,New Code,Borough,Names,Aged 0-15,Aged 16-64,Aged 65+,White,Mixed,Asian or Asian British,...,% Highest level of qualification: Level 2 qual...,% Highest level of qualification: Apprenticeship,% Highest level of qualification: Level 3 qual...,% Highest level of qualification: Level 4 qual...,% Highest level of qualification: Other qualif...,% of 16+ who are schoolchildren and full-time ...,% People with Bad or Very Bad Health,% People whose Day-to-day activities are limit...,NaN,2010.000000
1,00AA,E09000001,City of London,City of London,7.5,77.5,15,5799,289,940,...,6.6,0.7,7.2,68.4,6.2,6.2,3.4,4.4,7.640332,12.124906
2,00ABFX,E05000026,Barking and Dagenham,Abbey,25.27472527,69.96336996,5.128205128,3596,585,5619,...,11.5,1.1,8.5,34.5,16.7,14.3,4.3,5.1,7.897959,32.726127


In [39]:
# The first 4 columns seem to have a wrong name
df_atlas.rename(columns={'Unnamed: 0': 'Codes', 'Unnamed: 1': 'New Code', 'Unnamed: 2': 'Borough' , 'Unnamed: 3': 'Names'}, inplace=True)

In [40]:
df_atlas

,Codes,New Code,Borough,Names,Population and Age; Age structure (percentage) - 2013; Aged 0-15,Population and Age; Age structure (percentage) - 2013; Aged 16-64,Population and Age; Age structure (percentage) - 2013; Aged 65+,Diversity; Ethnic Group 5 groups - 2011 Census; White,Diversity; Ethnic Group 5 groups - 2011 Census; Mixed,Diversity; Ethnic Group 5 groups - 2011 Census; Asian or Asian British,...,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 2 qualifications,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Apprenticeship,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 3 qualifications,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 4 qualifications and above,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Other qualifications,Qualifications; Qualifications and Students - 2011 Census; % of 16+ who are schoolchildren and full-time students: Age 18 and over,Health and Disability; General Health - 2011 Census; % People with Bad or Very Bad Health,Health and Disability; General Health - 2011 Census; % People whose Day-to-day activities are limited a lot,"Happiness and Well-being; Subjective well-being average score, 2011/12 and 2012/13;",Indices of Deprivation; Average Score; 2010
0,Codes,New Code,Borough,Names,Aged 0-15,Aged 16-64,Aged 65+,White,Mixed,Asian or Asian British,...,% Highest level of qualification: Level 2 qual...,% Highest level of qualification: Apprenticeship,% Highest level of qualification: Level 3 qual...,% Highest level of qualification: Level 4 qual...,% Highest level of qualification: Other qualif...,% of 16+ who are schoolchildren and full-time ...,% People with Bad or Very Bad Health,% People whose Day-to-day activities are limit...,NaN,2010.000000
1,00AA,E09000001,City of London,City of London,7.5,77.5,15,5799,289,940,...,6.6,0.7,7.2,68.4,6.2,6.2,3.4,4.4,7.640332,12.124906
2,00ABFX,E05000026,Barking and Dagenham,Abbey,25.27472527,69.96336996,5.128205128,3596,585,5619,...,11.5,1.1,8.5,34.5,16.7,14.3,4.3,5.1,7.897959,32.726127
3,00ABFY,E05000027,Barking and Dagenham,Alibon,25.96153846,63.46153846,10.57692308,7275,362,743,...,15.7,1.8,9.3,16.7,8.9,5.8,6.7,8.8,7.358842,36.066225
4,00ABFZ,E05000028,Barking and Dagenham,Becontree,24.89626556,66.39004149,9.128630705,6789,524,2009,...,15.3,2,9.1,20.6,10,7.7,7,8.8,7.609123,31.686544
5,00ABGA,E05000029,Barking and Dagenham,Chadwell Heath,24.13793103,60.591133,15.27093596,6224,473,1560,...,14.9,2.2,10,19.5,8.3,5.3,6.8,9.8,7.108108,38.044504
6,00ABGB,E05000030,Barking and Dagenham,Eastbrook,20.28301887,65.09433962,14.62264151,7904,331,766,...,16.5,2.8,10,18.5,6.1,5,5.9,8.6,7.887917,25.674530
7,00ABGC,E05000031,Barking and Dagenham,Eastbury,25.64102564,64.52991453,9.829059829,6776,545,1826,...,13.9,1.9,8.9,20,10.7,7,6.4,8.2,8.112745,34.932636
8,00ABGD,E05000032,Barking and Dagenham,Gascoigne,31.67259786,62.63345196,5.693950178,4931,682,2574,...,12.6,1.2,9.3,25.2,13.8,10.3,5.9,7.1,7.820168,43.232252
9,00ABGE,E05000033,Barking and Dagenham,Goresbrook,25.66371681,63.27433628,10.61946903,7517,467,829,...,16.1,2.1,8.3,18.8,8.9,6,6.3,9.2,6.932432,34.326168


In [41]:
# the last three rows contain the demographic markers on country level and our not needed for our analysis
df_atlas.iloc[-3:]

,Codes,New Code,Borough,Names,Population and Age; Age structure (percentage) - 2013; Aged 0-15,Population and Age; Age structure (percentage) - 2013; Aged 16-64,Population and Age; Age structure (percentage) - 2013; Aged 65+,Diversity; Ethnic Group 5 groups - 2011 Census; White,Diversity; Ethnic Group 5 groups - 2011 Census; Mixed,Diversity; Ethnic Group 5 groups - 2011 Census; Asian or Asian British,...,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 2 qualifications,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Apprenticeship,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 3 qualifications,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 4 qualifications and above,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Other qualifications,Qualifications; Qualifications and Students - 2011 Census; % of 16+ who are schoolchildren and full-time students: Age 18 and over,Health and Disability; General Health - 2011 Census; % People with Bad or Very Bad Health,Health and Disability; General Health - 2011 Census; % People whose Day-to-day activities are limited a lot,"Happiness and Well-being; Subjective well-being average score, 2011/12 and 2012/13;",Indices of Deprivation; Average Score; 2010
626,#1,E12000007,NaN,London,19.82387476,68.81515745,11.36037479,4887435,405279,1511546,...,11.8,1.6,10.5,37.7,10,8.1,4.9,6.7,7.640332,NaN
627,#2,E92000001,NaN,England,18.95309227,63.77216928,17.27473845,45281142,1192879,4143403,...,15.22370749,3.565823564,12.35096053,27.37721571,5.726566087,5.462241816,5.5,8.3,NaN,NaN
628,#3,K04000001,NaN,England and Wales,18.89198229,64.06230146,17.04571626,48209395,1224400,4213531,...,15.3,3.6,12.3,27.2,5.7,5.5,5.6,8.5,NaN,NaN


In [42]:
# delete the last three rows
df_atlas.drop(df_atlas.index[-3:], inplace=True)
df_atlas

,Codes,New Code,Borough,Names,Population and Age; Age structure (percentage) - 2013; Aged 0-15,Population and Age; Age structure (percentage) - 2013; Aged 16-64,Population and Age; Age structure (percentage) - 2013; Aged 65+,Diversity; Ethnic Group 5 groups - 2011 Census; White,Diversity; Ethnic Group 5 groups - 2011 Census; Mixed,Diversity; Ethnic Group 5 groups - 2011 Census; Asian or Asian British,...,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 2 qualifications,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Apprenticeship,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 3 qualifications,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 4 qualifications and above,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Other qualifications,Qualifications; Qualifications and Students - 2011 Census; % of 16+ who are schoolchildren and full-time students: Age 18 and over,Health and Disability; General Health - 2011 Census; % People with Bad or Very Bad Health,Health and Disability; General Health - 2011 Census; % People whose Day-to-day activities are limited a lot,"Happiness and Well-being; Subjective well-being average score, 2011/12 and 2012/13;",Indices of Deprivation; Average Score; 2010
0,Codes,New Code,Borough,Names,Aged 0-15,Aged 16-64,Aged 65+,White,Mixed,Asian or Asian British,...,% Highest level of qualification: Level 2 qual...,% Highest level of qualification: Apprenticeship,% Highest level of qualification: Level 3 qual...,% Highest level of qualification: Level 4 qual...,% Highest level of qualification: Other qualif...,% of 16+ who are schoolchildren and full-time ...,% People with Bad or Very Bad Health,% People whose Day-to-day activities are limit...,NaN,2010.000000
1,00AA,E09000001,City of London,City of London,7.5,77.5,15,5799,289,940,...,6.6,0.7,7.2,68.4,6.2,6.2,3.4,4.4,7.640332,12.124906
2,00ABFX,E05000026,Barking and Dagenham,Abbey,25.27472527,69.96336996,5.128205128,3596,585,5619,...,11.5,1.1,8.5,34.5,16.7,14.3,4.3,5.1,7.897959,32.726127
3,00ABFY,E05000027,Barking and Dagenham,Alibon,25.96153846,63.46153846,10.57692308,7275,362,743,...,15.7,1.8,9.3,16.7,8.9,5.8,6.7,8.8,7.358842,36.066225
4,00ABFZ,E05000028,Barking and Dagenham,Becontree,24.89626556,66.39004149,9.128630705,6789,524,2009,...,15.3,2,9.1,20.6,10,7.7,7,8.8,7.609123,31.686544
5,00ABGA,E05000029,Barking and Dagenham,Chadwell Heath,24.13793103,60.591133,15.27093596,6224,473,1560,...,14.9,2.2,10,19.5,8.3,5.3,6.8,9.8,7.108108,38.044504
6,00ABGB,E05000030,Barking and Dagenham,Eastbrook,20.28301887,65.09433962,14.62264151,7904,331,766,...,16.5,2.8,10,18.5,6.1,5,5.9,8.6,7.887917,25.674530
7,00ABGC,E05000031,Barking and Dagenham,Eastbury,25.64102564,64.52991453,9.829059829,6776,545,1826,...,13.9,1.9,8.9,20,10.7,7,6.4,8.2,8.112745,34.932636
8,00ABGD,E05000032,Barking and Dagenham,Gascoigne,31.67259786,62.63345196,5.693950178,4931,682,2574,...,12.6,1.2,9.3,25.2,13.8,10.3,5.9,7.1,7.820168,43.232252
9,00ABGE,E05000033,Barking and Dagenham,Goresbrook,25.66371681,63.27433628,10.61946903,7517,467,829,...,16.1,2.1,8.3,18.8,8.9,6,6.3,9.2,6.932432,34.326168


In [43]:
# row 0 od df_atlas seems to have column names and years in it, therefore, drop it
df_atlas.drop(df_atlas.index[0], inplace=True)
df_atlas

,Codes,New Code,Borough,Names,Population and Age; Age structure (percentage) - 2013; Aged 0-15,Population and Age; Age structure (percentage) - 2013; Aged 16-64,Population and Age; Age structure (percentage) - 2013; Aged 65+,Diversity; Ethnic Group 5 groups - 2011 Census; White,Diversity; Ethnic Group 5 groups - 2011 Census; Mixed,Diversity; Ethnic Group 5 groups - 2011 Census; Asian or Asian British,...,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 2 qualifications,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Apprenticeship,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 3 qualifications,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Level 4 qualifications and above,Qualifications; Qualifications and Students - 2011 Census; % Highest level of qualification: Other qualifications,Qualifications; Qualifications and Students - 2011 Census; % of 16+ who are schoolchildren and full-time students: Age 18 and over,Health and Disability; General Health - 2011 Census; % People with Bad or Very Bad Health,Health and Disability; General Health - 2011 Census; % People whose Day-to-day activities are limited a lot,"Happiness and Well-being; Subjective well-being average score, 2011/12 and 2012/13;",Indices of Deprivation; Average Score; 2010
1,00AA,E09000001,City of London,City of London,7.5,77.5,15,5799,289,940,...,6.6,0.7,7.2,68.4,6.2,6.2,3.4,4.4,7.640332,12.124906
2,00ABFX,E05000026,Barking and Dagenham,Abbey,25.27472527,69.96336996,5.128205128,3596,585,5619,...,11.5,1.1,8.5,34.5,16.7,14.3,4.3,5.1,7.897959,32.726127
3,00ABFY,E05000027,Barking and Dagenham,Alibon,25.96153846,63.46153846,10.57692308,7275,362,743,...,15.7,1.8,9.3,16.7,8.9,5.8,6.7,8.8,7.358842,36.066225
4,00ABFZ,E05000028,Barking and Dagenham,Becontree,24.89626556,66.39004149,9.128630705,6789,524,2009,...,15.3,2,9.1,20.6,10,7.7,7,8.8,7.609123,31.686544
5,00ABGA,E05000029,Barking and Dagenham,Chadwell Heath,24.13793103,60.591133,15.27093596,6224,473,1560,...,14.9,2.2,10,19.5,8.3,5.3,6.8,9.8,7.108108,38.044504
6,00ABGB,E05000030,Barking and Dagenham,Eastbrook,20.28301887,65.09433962,14.62264151,7904,331,766,...,16.5,2.8,10,18.5,6.1,5,5.9,8.6,7.887917,25.674530
7,00ABGC,E05000031,Barking and Dagenham,Eastbury,25.64102564,64.52991453,9.829059829,6776,545,1826,...,13.9,1.9,8.9,20,10.7,7,6.4,8.2,8.112745,34.932636
8,00ABGD,E05000032,Barking and Dagenham,Gascoigne,31.67259786,62.63345196,5.693950178,4931,682,2574,...,12.6,1.2,9.3,25.2,13.8,10.3,5.9,7.1,7.820168,43.232252
9,00ABGE,E05000033,Barking and Dagenham,Goresbrook,25.66371681,63.27433628,10.61946903,7517,467,829,...,16.1,2.1,8.3,18.8,8.9,6,6.3,9.2,6.932432,34.326168
10,00ABGF,E05000034,Barking and Dagenham,Heath,25.57077626,62.55707763,11.87214612,7304,364,711,...,15.1,2.4,8.8,17.6,7.4,6,6.7,10.2,7.471154,38.757016


In [44]:
# can delete column `Codes` since this represents old area codes, the tesco data set is indexed by `New Code`
df_atlas = df_atlas.drop(['Codes'], axis=1)

In [45]:
df_atlas.columns.values

array(['New Code', 'Borough', 'Names',
       'Population and Age; Age structure (percentage) - 2013; Aged 0-15',
       'Population and Age; Age structure (percentage) - 2013; Aged 16-64',
       'Population and Age; Age structure (percentage) - 2013; Aged 65+',
       'Diversity; Ethnic Group 5 groups - 2011 Census; White',
       'Diversity; Ethnic Group 5 groups - 2011 Census; Mixed',
       'Diversity; Ethnic Group 5 groups - 2011 Census; Asian or Asian British',
       'Diversity; Ethnic Group 5 groups - 2011 Census; Black or Black British',
       'Diversity; Ethnic Group 5 groups - 2011 Census; Other',
       'Diversity; Religion - 2011 Census; Christian',
       'Diversity; Religion - 2011 Census; Buddhist',
       'Diversity; Religion - 2011 Census; Hindu',
       'Diversity; Religion - 2011 Census; Jewish',
       'Diversity; Religion - 2011 Census; Muslim',
       'Diversity; Religion - 2011 Census; Sikh',
       'Diversity; Religion - 2011 Census; Other religions',
       

In [46]:
# very long column names, rename columns to more concise names
import re
rename_map = {name: re.sub('.*; ', '', name) for name in df_atlas.columns.values}
# after inspection 'Household Income; Median Modelled Household income (£); 2012/13',
# 'Household Income; Mean Modelled Household income (£); 2012/13',
# 'Happiness and Well-being; Subjective well-being average score, 2011/12 and 2012/13; ',
# and 'Indices of Deprivation; Average Score; 2010' have been replaced too much, manually add those column names
rename_map['Household Income; Median Modelled Household income (£); 2012/13'] = 'Household Income Mean'
rename_map['Household Income; Mean Modelled Household income (£); 2012/13'] = 'Household Income Median'
rename_map['Happiness and Well-being; Subjective well-being average score, 2011/12 and 2012/13; '] = 'Well-Being'
rename_map['Indices of Deprivation; Average Score; 2010'] = 'IOD AVG'
df_atlas.rename(columns=rename_map, inplace=True)
                         
df_atlas.rename(columns={'% No qualifications': 'No qualifications', 
                         '% Highest level of qualification: Level 1 qualifications': 'Level 1 qualifications', 
                         '% Highest level of qualification: Level 2 qualifications': 'Level 2 qualifications', 
                         '% Highest level of qualification: Apprenticeship': 'Apprenticeship qualifications', 
                         '% Highest level of qualification: Level 3 qualifications': 'Level 3 qualifications', 
                         '% Highest level of qualification: Level 4 qualifications and above': 'Level >=4 qualifications',
                         '% Highest level of qualification: Other qualifications': 'Other qualifications', 
                         '% People with Bad or Very Bad Health': 'Bad Health',
                         '% People whose Day-to-day activities are limited a lot': 'Limited activities'},
                inplace=True)

In [47]:
# checking column type
df_atlas.dtypes

New Code                                                                    object
Borough                                                                     object
Names                                                                       object
Aged 0-15                                                                   object
Aged 16-64                                                                  object
Aged 65+                                                                    object
White                                                                       object
Mixed                                                                       object
Asian or Asian British                                                      object
Black or Black British                                                      object
Other                                                                       object
Christian                                                                   object
Budd

The values in the columns seem to be converted to objects instead of floats or ints, therefore convert these objects to numveric values where possible. None of these columns store string data (except the first three columns).

In [48]:
# to numeric conversion
df_atlas.loc[:, 'Aged 0-15':] = df_atlas.loc[:, 'Aged 0-15':].apply(pd.to_numeric, errors='coerce')
df_atlas.dtypes

New Code                                                                    object
Borough                                                                     object
Names                                                                       object
Aged 0-15                                                                  float64
Aged 16-64                                                                 float64
Aged 65+                                                                   float64
White                                                                        int64
Mixed                                                                        int64
Asian or Asian British                                                       int64
Black or Black British                                                       int64
Other                                                                        int64
Christian                                                                    int64
Budd

In [49]:
# checking for NA values
np.sum(df_atlas.isna())

New Code                                                                   0
Borough                                                                    0
Names                                                                      0
Aged 0-15                                                                  0
Aged 16-64                                                                 0
Aged 65+                                                                   0
White                                                                      0
Mixed                                                                      0
Asian or Asian British                                                     0
Black or Black British                                                     0
Other                                                                      0
Christian                                                                  0
Buddhist                                                                   0

In [50]:
age_cols = ['Aged 0-15', 'Aged 16-64', 'Aged 65+'] #sum to 1
ethnicities_cols = ['White', 'Mixed', 'Asian or Asian British', 'Black or Black British', 'Other'] #sum to 1
religion_cols = ['Christian', 'Buddhist', 'Hindu', 'Jewish', 'Muslim', 'Sikh',
       'No religion', 'Other religions', 'Religion not stated'] # sum to 1
born_cols = ['Born in UK', 'Not Born in UK'] # sum to 1
wealth_cols = ['Household Income Median', 'IOD AVG'] #'Well-Being'
qualilication_cols = ['No qualifications', 'Level 1 qualifications', 'Level 2 qualifications',
       'Apprenticeship qualifications', 'Level 3 qualifications',
       'Level >=4 qualifications', 'Other qualifications'] # sum to 1 
disability_cols = ['Bad Health', 'Limited activities']
all_features = age_cols + ethnicities_cols + religion_cols + born_cols + wealth_cols + qualilication_cols + disability_cols

In [51]:
def normalize(df, cols):
    total = df[cols].sum(axis=1)
    df[cols] = df[cols].div(total, axis=0)
    return df

def standartize(df, cols):
    means = df[cols].mean()
    stds = df[cols].std()
    df[cols] = (df[cols] - means) / stds
    return df

def min_max_scaling(df, cols):
    mins = df[cols].min()
    maxs = df[cols].max()
    df[cols] = (df[cols] - mins) / (maxs - mins)
    return df

In [52]:
#normalize
df_atlas[age_cols] /= 100
df_atlas = normalize(df_atlas, ethnicities_cols)
df_atlas = normalize(df_atlas, religion_cols)
df_atlas = normalize(df_atlas, born_cols)
df_atlas[qualilication_cols] /= 100

In [53]:
df_atlas['Household Income Median'] = df_atlas['Household Income Median'].apply(np.log)
df_atlas = standartize(df_atlas, wealth_cols)

## Tesco groceries preprocessing

In [54]:
important_cols = ['weight', 'volume', 'fat', 'saturate', 'salt', 'sugar', 'protein', 'carb',
                 'fibre', 'alcohol', 'energy_fat', 'energy_saturate', 'energy_sugar', 'energy_protein',
                 'energy_carb', 'energy_fibre', 'energy_alcohol', 'energy_tot', 'f_energy_fat', 'f_energy_saturate', 
                 'f_energy_sugar', 'f_energy_protein', 'f_energy_carb', 'f_energy_fibre', 'f_energy_alcohol', 
                 'energy_density', 'h_nutrients_weight', 'h_nutrients_weight_norm', 'h_nutrients_calories',
                  'h_nutrients_calories_norm', 'representativeness_norm']

In [55]:
df_atlas.set_index('New Code', drop=True, inplace=True)

In [56]:
# merge the data sets by ward
df_ward_atlas = pd.merge(df_ward[important_cols], df_atlas, how='inner', left_index=True, right_index=True, validate='1:1')

## Save dataset

In [58]:
df_ward_atlas.head(1)

,weight,volume,fat,saturate,salt,sugar,protein,carb,fibre,alcohol,...,Level 2 qualifications,Apprenticeship qualifications,Level 3 qualifications,Level >=4 qualifications,Other qualifications,% of 16+ who are schoolchildren and full-time students: Age 18 and over,Bad Health,Limited activities,Well-Being,IOD AVG
E05000026,449.535137,125.960833,9.488797,3.693721,0.58324,10.966213,4.97756,19.381951,1.564721,0.198172,...,0.115,0.011,0.085,0.345,0.167,14.3,4.3,5.1,7.897959,0.694138


In [61]:
PREPROCESSED_PATH = 'preprocessed'
DATASET_NAME='df_ward_atlas.csv'
DATASET_PATH = join(FOLDER_PATH, PREPROCESSED_PATH, DATASET_NAME)
df_ward_atlas.to_csv(DATASET_PATH)